# new model

torch 모델을 다른것을 사용해서 확인하기에, 새로운 그리드 파일을 연것.

encoder, transformer 이미지를 테스트 예정.

encoder 이미지는 1개라서 그리드는 필요없음.

In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import math



p = os.path.abspath('../') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# data 분류

In [3]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('D:/project/voice_pathology_ai/voice_data/all_data/pathology/phrase/*.wav')
healthy = glob('D:/project/voice_pathology_ai/voice_data/all_data/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1354:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  1354
Healthy:  634
총 데이터수 :  1988
---
훈련 셋 :  1590 Counter({'pathology': 1083, 'healthy': 507})
테스트 셋 :  398 Counter({'pathology': 271, 'healthy': 127})
---


In [4]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 



In [5]:
#2. trainset에만 random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res




 fold0 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold1 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold2 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold3 
before dataset shape Counter({'pathology': 867, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 867, 'healthy': 867})

 fold4 
before dataset shape Counter({'pathology': 867, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 867, 'healthy': 867})


In [6]:
import pickle

 
#load
with open("../../voice_data/all_data/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

# data set

In [7]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

In [8]:
# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes

        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

# data loader

In [9]:
#3. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [10]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.

#확인을 위해 데이터셋 하나만 확인



train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_train_list,
                                                   classes,
                                               data_num=0,
                                               training=True
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.

validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               data_num=0,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 



In [11]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# model

https://pytorch.org/audio/main/tutorials/speech_recognition_pipeline_tutorial.html


dat1=torch.randn((32,12,318,768))
model1=nn.Conv2d(12,3,(3,3))
dat2=model1(dat1)
print(dat2.size())
#torch.Size([32, 3, 316, 766])
res_18_sample = models.resnet18(pretrained=True)
dat3=res_18_sample(dat2)
print(dat3.size())



----

In [13]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [12]:

#mean 변경 및 데이터 변경 실험
        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.sum(axis=1)/(hidden_feature!=0).sum(axis=1)### 수정 zero 포함않게.
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [27]:
###
# model for enocoder

sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(512,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(256,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        x = encoder_x
        #x = self.zero_padding(x,encoder_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [ ]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.


bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model_wav2vec = bundle.get_model().to(DEVICE)

print(model_wav2vec.__class__)

        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        self.model_wav2vec = bundle.get_model().to(DEVICE)
        self.res_18 = models.resnet18(pretrained=True)
        self.conv_model1=nn.Conv2d(12,3,(3,3))
        
        #input_dim =101951
        
        self.fc = nn.Sequential(nn.Linear(1000,512),
                                nn.BatchNorm1d(512),
                                nn.ReLU(),
                                nn.Linear(512,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Linear(256,50),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=[ feat.mean(axis=1) for feat  in  hidden_feature]
        return hidden_feature
        
        
    def forward(self,x):
        with torch.inference_mode():
            x,_ = self.model_wav2vec.extract_features(x)
        x = torch.stack(x,axis=1)
        x = self.conv_model1(x)
        x = self.res_18(x)
        x= self.fc(x)
    
        return x 
    


In [15]:
#7. Optimizer, Objective Function
def model_initialize(num):
    model = wav_path_Net(num).to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [14]:
#7. Optimizer, Objective Function
def model_initialize():
    model = wav_path_Net().to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [20]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label,sig_len) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        sig_len = sig_len.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image,sig_len) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy


In [21]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            
            #true.false값을 sum해줌. item
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy


In [22]:
#데이터 로더 제작 함수

def load_data(data_ind):

    train_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_train_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=True
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,
                                               ) # 순서가 암기되는것을 막기위해.

    validation_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_valid_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=False
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 
    return train_loader,validation_loader



In [23]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def test_evaluate(model,test_loader):
    model.eval()
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            output = F.softmax(output, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
        return predictions,answers

In [24]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    
    
    for grid_num in range(0,24):
        
        average_accuracy = 0
        average_fscore = 0
        for data_ind in range(1,6):
            model=model_initialize(grid_num)
            check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(grid_num)+'-'+str(data_ind)+'-alldata-phrase.pt'
            model.load_state_dict(torch.load(check_path))

            predictions,answers = test_evaluate(model, test_loader)
            predictions=[ dat.cpu().numpy() for dat in predictions]
            answers=[ dat.cpu().numpy() for dat in answers]


            cf = confusion_matrix(answers, predictions)
            cf_list.append(cf)

            acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
            average_accuracy+=acc
            precision=cf[0,0]/(cf[0,0]+cf[1,0])
            recall=cf[0,0]/(cf[0,0]+cf[0,1])
            #fscore=2*precision*recall/(precision+recall)

            #fscroe macro추가
            fscore = f1_score(answers,predictions,average='macro')
            average_fscore+=fscore
            

            print('{}번 모델 / {}번 fold'.format(grid_num+1,data_ind))
            print("Accuracy : {:.4f}% ".format(acc*100))
            print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
            print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
            print("f score : {:.4f} ".format(fscore))
            print(cf)
            print("-----")
        print("----")
        print("* {}번 모델 {:.4f}% / score: {:.4f}".format(grid_num+1, 100*average_accuracy/5,average_fscore/5))
        print("----")


In [31]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    for data_ind in range(1,2):
        model=model_initialize(grid_num)
        check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'-alldata-phrase.pt'
        model.load_state_dict(torch.load(check_path))

        predictions,answers = test_evaluate(model, test_loader)
        predictions=[ dat.cpu().numpy() for dat in predictions]
        answers=[ dat.cpu().numpy() for dat in answers]


        cf = confusion_matrix(answers, predictions)
        cf_list.append(cf)

        acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
        precision=cf[0,0]/(cf[0,0]+cf[1,0])
        recall=cf[0,0]/(cf[0,0]+cf[0,1])
        #fscore=2*precision*recall/(precision+recall)

        #fscroe macro추가
        fscore = f1_score(answers,predictions,average='macro')

        print('{}번 모델'.format(grid_num+1))
        print("Accuracy : {:.4f}% ".format(acc*100))
        print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
        print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
        print("f score : {:.4f} ".format(fscore))
        print(cf)
        print("-----")




In [28]:
#10. 학습 및 평가.
# kfold 적용
train_accs = []
valid_accs = []
for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'-alldata-phrase.pt'
    print(check_path)
    early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
    train_loader,validation_loader = load_data(data_ind-1)

    best_train_acc=0 # accuracy 기록용
    best_valid_acc=0

    model=model_initialize()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


    print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
    for Epoch in range(1,EPOCHS+1):
        train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
        valid_loss,valid_accuracy = evaluate(model, validation_loader)


        print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
              format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))


        early_stopping(valid_loss, model)
        if -early_stopping.best_score == valid_loss:
            best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

        if early_stopping.early_stop:
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
                print("[{} 교차검증] Early stopping".format(data_ind))
                break

        if Epoch==EPOCHS:
            #만약 early stop 없이 40 epoch라서 중지 된 경우.
            train_accs.append(best_train_acc)
            valid_accs.append(best_valid_acc)

../checkpoint/checkpoint_w2v_large_model60k_encoder-1.pt


TypeError: __init__() missing 1 required positional argument: 'num'

In [ ]:
#10. 학습 및 평가.
# kfold 적용



all_train_accs =[]
all_valid_accs =[]


for num in range(24):
    print("{}-번 레이어".format(num))
    train_accs = []
    valid_accs = []

    for data_ind in range(1,6):

        check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(num)+'-'+str(data_ind)+'-alldata-phrase.pt'
        print(check_path)
        early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
        train_loader,validation_loader = load_data(data_ind-1)

        best_train_acc=0 # accuracy 기록용
        best_valid_acc=0

        model=model_initialize(num)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


        print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
        for Epoch in range(1,EPOCHS+1):
            train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
            valid_loss,valid_accuracy = evaluate(model, validation_loader)


            print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
                  format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
            

            early_stopping(valid_loss, model)
            if -early_stopping.best_score == valid_loss:
                best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

            if early_stopping.early_stop:
                    train_accs.append(best_train_acc)
                    valid_accs.append(best_valid_acc)
                    print("[{} 교차검증] Early stopping".format(data_ind))
                    break

            if Epoch==EPOCHS:
                #만약 early stop 없이 40 epoch라서 중지 된 경우.
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
    all_train_accs.append(train_accs)
    all_valid_accs.append(valid_accs)

0-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_0-1-alldata-phrase.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0163	 Train Acc:74.71 %  | 	Valid Loss:0.0138 	Valid Acc: 81.45 %

Validation loss decreased (inf --> 0.013843).  Saving model ...

[EPOCH:2]	 Train Loss:0.0142	 Train Acc:81.47 %  | 	Valid Loss:0.0131 	Valid Acc: 81.13 %

Validation loss decreased (0.013843 --> 0.013133).  Saving model ...

[EPOCH:3]	 Train Loss:0.0121	 Train Acc:83.37 %  | 	Valid Loss:0.0129 	Valid Acc: 81.45 %

Validation loss decreased (0.013133 --> 0.012898).  Saving model ...

[EPOCH:4]	 Train Loss:0.0125	 Train Acc:83.78 %  | 	Valid Loss:0.0128 	Valid Acc: 81.76 %

Validation loss decreased (0.012898 --> 0.012757).  Saving model ...

[EPOCH:5]	 Train Loss:0.0114	 Train Acc:85.80 %  | 	Valid Loss:0.0120 	Valid Acc: 82.39 %

Validation loss decreased (0.012757 --> 0.012019).  Saving model ...

[EPOCH:6]	 Train Loss:0.0105	 Train Acc:86.20 %  | 	Valid Loss:0.0127 	Valid Acc: 78.62 %

EarlyStoppi

# test

In [20]:
for num, valid_fold in enumerate(all_valid_accs):
    print("-----\n")
    avg_train = []
    avg_valid = []
    for num_fold,valid in enumerate(valid_fold):
        print("{}번 모델. {} 폴드.  train :{:.4f}/ valid: {:.4f}".format(num+1,num_fold+1,all_train_accs[num][num_fold],valid) )
        avg_train.append(all_train_accs[num][num_fold])
        avg_valid.append(valid)
    print("* 평균 train acc : {:.4f}% / 평균 valid acc : {:.4f}% ".format(np.array(avg_train).mean(), np.array(avg_valid).mean()) )

-----

1번 모델. 1 폴드.  train :87.0670/ valid: 83.0189
1번 모델. 2 폴드.  train :87.1824/ valid: 86.7925
1번 모델. 3 폴드.  train :73.4411/ valid: 82.0755
1번 모델. 4 폴드.  train :81.6032/ valid: 83.3333
1번 모델. 5 폴드.  train :88.6967/ valid: 86.4780
* 평균 train acc : 83.5981% / 평균 valid acc : 84.3396% 
-----

2번 모델. 1 폴드.  train :87.8176/ valid: 84.5912
2번 모델. 2 폴드.  train :89.7806/ valid: 87.4214
2번 모델. 3 폴드.  train :85.4503/ valid: 83.3333
2번 모델. 4 폴드.  train :87.6586/ valid: 85.8491
2번 모델. 5 폴드.  train :88.0046/ valid: 83.6478
* 평균 train acc : 87.7423% / 평균 valid acc : 84.9686% 
-----

3번 모델. 1 폴드.  train :87.9330/ valid: 84.9057
3번 모델. 2 폴드.  train :87.0092/ valid: 88.0503
3번 모델. 3 폴드.  train :88.5681/ valid: 83.6478
3번 모델. 4 폴드.  train :87.0819/ valid: 85.8491
3번 모델. 5 폴드.  train :88.8120/ valid: 86.4780
* 평균 train acc : 87.8809% / 평균 valid acc : 85.7862% 
-----

4번 모델. 1 폴드.  train :87.1824/ valid: 81.7610
4번 모델. 2 폴드.  train :85.3349/ valid: 86.7925
4번 모델. 3 폴드.  train :86.5473/ valid: 84.5912
4번 

In [22]:
for num,valid in enumerate(valid_accs):
    print("{}번 모델. train :{:.4f}/ valid: {:.4f}".format(num+1,train_accs[num],valid) )

1번 모델. train :84.4037/ valid: 87.7133
2번 모델. train :84.7313/ valid: 87.3720
3번 모델. train :86.1730/ valid: 88.3959
4번 모델. train :88.7287/ valid: 87.0307
5번 모델. train :82.9620/ valid: 83.2765
6번 모델. train :86.4351/ valid: 86.3481
7번 모델. train :86.6317/ valid: 87.3720
8번 모델. train :86.3041/ valid: 86.6894
9번 모델. train :86.8283/ valid: 87.0307
10번 모델. train :86.5662/ valid: 86.6894
11번 모델. train :86.3041/ valid: 87.7133
12번 모델. train :87.5491/ valid: 86.0068
13번 모델. train :86.6972/ valid: 83.6177
14번 모델. train :85.8453/ valid: 81.9113
15번 모델. train :81.1271/ valid: 86.0068
16번 모델. train :83.2241/ valid: 84.3003
17번 모델. train :84.3381/ valid: 85.6655
18번 모델. train :85.8453/ valid: 84.3003
19번 모델. train :81.1271/ valid: 83.9590
20번 모델. train :84.0105/ valid: 84.6416
21번 모델. train :83.8794/ valid: 86.0068
22번 모델. train :53.3421/ valid: 65.1877
23번 모델. train :53.9974/ valid: 65.5290
24번 모델. train :55.4391/ valid: 60.4096


In [23]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_transformer_all.csv")

In [ ]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_encoder.csv")

In [ ]:
#encoder layer
test_grid()

In [21]:
# 24 레이어
test_grid()

1번 모델 / 1번 fold
Accuracy : 83.9196% 
Precision (pathology 예측한 것중 맞는 것) : 0.8996
recall (실제 pathology 중  예측이 맞는 것) : 0.8598
f score : 0.8193 
[[233  38]
 [ 26 101]]
-----
1번 모델 / 2번 fold
Accuracy : 80.9045% 
Precision (pathology 예측한 것중 맞는 것) : 0.8824
recall (실제 pathology 중  예측이 맞는 것) : 0.8303
f score : 0.7870 
[[225  46]
 [ 30  97]]
-----
1번 모델 / 3번 fold
Accuracy : 83.6683% 
Precision (pathology 예측한 것중 맞는 것) : 0.8815
recall (실제 pathology 중  예측이 맞는 것) : 0.8782
f score : 0.8125 
[[238  33]
 [ 32  95]]
-----
1번 모델 / 4번 fold
Accuracy : 82.1608% 
Precision (pathology 예측한 것중 맞는 것) : 0.9065
recall (실제 pathology 중  예측이 맞는 것) : 0.8229
f score : 0.8041 
[[223  48]
 [ 23 104]]
-----
1번 모델 / 5번 fold
Accuracy : 81.9095% 
Precision (pathology 예측한 것중 맞는 것) : 0.8964
recall (실제 pathology 중  예측이 맞는 것) : 0.8303
f score : 0.7996 
[[225  46]
 [ 26 101]]
-----
----
* 1번 모델 82.5126% / score: 0.8045
----
2번 모델 / 1번 fold
Accuracy : 82.6633% 
Precision (pathology 예측한 것중 맞는 것) : 0.8915
recall (실제 pathology 중  예측이

10번 모델 / 2번 fold
Accuracy : 80.9045% 
Precision (pathology 예측한 것중 맞는 것) : 0.9079
recall (실제 pathology 중  예측이 맞는 것) : 0.8007
f score : 0.7926 
[[217  54]
 [ 22 105]]
-----
10번 모델 / 3번 fold
Accuracy : 80.1508% 
Precision (pathology 예측한 것중 맞는 것) : 0.9211
recall (실제 pathology 중  예측이 맞는 것) : 0.7749
f score : 0.7878 
[[210  61]
 [ 18 109]]
-----
10번 모델 / 4번 fold
Accuracy : 82.4121% 
Precision (pathology 예측한 것중 맞는 것) : 0.8880
recall (실제 pathology 중  예측이 맞는 것) : 0.8487
f score : 0.8024 
[[230  41]
 [ 29  98]]
-----
10번 모델 / 5번 fold
Accuracy : 80.9045% 
Precision (pathology 예측한 것중 맞는 것) : 0.9046
recall (실제 pathology 중  예측이 맞는 것) : 0.8044
f score : 0.7920 
[[218  53]
 [ 23 104]]
-----
----
* 10번 모델 80.8543% / score: 0.7910
----
11번 모델 / 1번 fold
Accuracy : 80.4020% 
Precision (pathology 예측한 것중 맞는 것) : 0.9106
recall (실제 pathology 중  예측이 맞는 것) : 0.7897
f score : 0.7884 
[[214  57]
 [ 21 106]]
-----
11번 모델 / 2번 fold
Accuracy : 82.4121% 
Precision (pathology 예측한 것중 맞는 것) : 0.9170
recall (실제 pathology

19번 모델 / 3번 fold
Accuracy : 77.8894% 
Precision (pathology 예측한 것중 맞는 것) : 0.9178
recall (실제 pathology 중  예측이 맞는 것) : 0.7417
f score : 0.7664 
[[201  70]
 [ 18 109]]
-----
19번 모델 / 4번 fold
Accuracy : 80.1508% 
Precision (pathology 예측한 것중 맞는 것) : 0.9211
recall (실제 pathology 중  예측이 맞는 것) : 0.7749
f score : 0.7878 
[[210  61]
 [ 18 109]]
-----
19번 모델 / 5번 fold
Accuracy : 79.3970% 
Precision (pathology 예측한 것중 맞는 것) : 0.9091
recall (실제 pathology 중  예측이 맞는 것) : 0.7749
f score : 0.7789 
[[210  61]
 [ 21 106]]
-----
----
* 19번 모델 78.7437% / score: 0.7741
----
20번 모델 / 1번 fold
Accuracy : 78.6432% 
Precision (pathology 예측한 것중 맞는 것) : 0.9152
recall (실제 pathology 중  예측이 맞는 것) : 0.7565
f score : 0.7729 
[[205  66]
 [ 19 108]]
-----
20번 모델 / 2번 fold
Accuracy : 79.8995% 
Precision (pathology 예측한 것중 맞는 것) : 0.9064
recall (실제 pathology 중  예측이 맞는 것) : 0.7860
f score : 0.7830 
[[213  58]
 [ 22 105]]
-----
20번 모델 / 3번 fold
Accuracy : 76.6332% 
Precision (pathology 예측한 것중 맞는 것) : 0.9159
recall (실제 pathology

In [ ]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_'+str(data_ind)+'.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))


